In [6]:
import dotenv
import json 
import operator
from pydantic import BaseModel, Field
from typing import List, Annotated
from typing_extensions import TypedDict
from langgraph.constants import Send
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

dotenv.load_dotenv(dotenv_path=".env")

True

In [21]:
with open('data/gold/stats-2023.json', 'r') as file:
    statistics_document = json.load(file)

analyst_report_prompt = ChatPromptTemplate.from_template(
    """You are an {analyst_topic} AI analyst. 
            
Your task is to create a short, easily digestible section of a report based on your analysis of your e-commerce company's customer feedback statistics json document provided to you.

1. Analyze the content of the source document:

{statistics_document}

2. Create a report for the CEO focusing on the most relevant numbers to your area of interest ({analyst_topic}), structure it using markdown formatting:
- Use ## for the section title
- Use ### for sub-section headers
- Identify up to 3 positive and negative insights related to {analyst_topic}
- It should be descriptive, don't propose solutions 
- Use the gathered data as your source

4. Make your title and content engaging based upon the focus area of the analyst: 
{analyst_topic}

5. End with a summary section:
- Set up summary with general background / context related to the focus area of the analyst
- Emphasize what is novel, interesting, or surprising about insights gathered from the data
- Aim for approximately 400 words maximum
        
6. Final review:
- Ensure the report follows the required structure
- Include no preamble before the title of the report
- Check that all guidelines have been followed"""
)

gpt4o_llm = ChatOpenAI(model="gpt-4o", temperature=0)

analyst_report_chain = analyst_report_prompt | gpt4o_llm

In [22]:
analysts_reports = {
    "logistics": analyst_report_chain.invoke(
        {
            "statistics_document": statistics_document,
            "analyst_topic": "logistics"
        }
    ),
    "branding": analyst_report_chain.invoke(
        {
            "statistics_document": statistics_document,
            "analyst_topic": "branding"
        }
    ),
    "manufacture": analyst_report_chain.invoke(
        {
            "statistics_document": statistics_document,
            "analyst_topic": "manufacture"
        }
    ),
    "sales": analyst_report_chain.invoke(
        {
            "statistics_document": statistics_document,
            "analyst_topic": "sales"
        }
    )
}

In [26]:
for analyst, report in analysts_reports.items():
    f = open(f"reports/{analyst}.md", "w")
    f.write(report.content)
    f.close()